In [1]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 18,Total memory: 49.01 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45513,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:39653,Total threads: 3
Dashboard: http://127.0.0.1:36447/status,Memory: 8.17 GiB
Nanny: tcp://127.0.0.1:43509,


In [1]:
import dask.dataframe as dd

df = dd.read_csv("/mnt/abnorm/abnorm/*.csv", blocksize="256MB")
df.head()

,when,hwid,metric,value
0,1601510485159,SW-065,SA4,0
1,1601510485159,SW-065,SA3,0
2,1601510485159,SW-065,SA2,0
3,1601510485159,SW-065,S34,0
4,1601510485159,SW-065,S33,1


In [3]:
df["when"].min().compute(), df["when"].max().compute()

(np.int64(1601510422405), np.int64(1617235197606))

In [2]:
# Convert UNIX milliseconds → datetime
df["timestamp"] = dd.to_datetime(df["when"], unit="ms", errors="coerce")
df = df.dropna(subset=["timestamp"])

#  Filter to Jan–Mar 2021
df = df[(df["timestamp"] >= "2021-01-01") & (df["timestamp"] <= "2021-03-31")]

#  Set index & repartition by day
df = df.set_index("timestamp")
df = df.repartition(freq="1D")

df.head()

,when,hwid,metric,value
timestamp,,,,
2021-01-01 00:00:01.846,1609459201846,SW-088,S108,0
2021-01-01 00:00:01.846,1609459201846,SW-088,S83,77
2021-01-01 00:00:01.846,1609459201846,SW-088,S81,124
2021-01-01 00:00:01.846,1609459201846,SW-088,S112,1
2021-01-01 00:00:01.846,1609459201846,SW-088,S97,89


In [3]:
engine_metrics = ["S117", "S118", "S169", "S170"]

df_eng = df[df["metric"].isin(engine_metrics)]
df_eng.head()

,when,hwid,metric,value
timestamp,,,,
2021-01-01 00:00:01.846,1609459201846,SW-088,S118,0
2021-01-01 00:00:01.846,1609459201846,SW-088,S117,0
2021-01-01 00:00:04.289,1609459204289,SW-088,S169,0
2021-01-01 00:00:04.289,1609459204289,SW-088,S170,0
2021-01-01 00:00:28.352,1609459228352,SW-065,S117,0


In [7]:
df_eng["value"].describe().compute()

count    1.965996e+06
mean     2.078056e-01
std      4.057370e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: value, dtype: float64